In [1]:
import pandas as pd 

In [3]:
data= pd.read_csv('merged_telemetry_train.csv')

## the nural netwrok

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf

# Sample DataFrame creation for demonstration
data = pd.read_csv('merged_telemetry_train.csv')

# Check for missing values and ensure correct type
data.dropna(inplace=True)
data['operation_kind_id'] = data['operation_kind_id'].astype(int)
# Adjust labels to fit into the range [0, 4]
data['operation_kind_id'] = data['operation_kind_id'].replace(5, 4)

# Feature selection
features = ['mdm_object_name', 'mdm_model_id', 'lon', 'lat', 'alt', 
            'speed_gps', 'direction', 'accel_forward_nn', 'accel_braking_nn', 
            'accel_angular_nn', 'accel_vertical_nn', 'speed_change', 'distance']
X = data[features].values
y = data['operation_kind_id'].values

# Normalize features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Define a simple model
def create_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')  # 5 classes
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create and train the model
model = create_model(input_shape=(X_train.shape[1],))
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))


Epoch 1/50
10221/10221 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.6467 - loss: 0.9050 - val_accuracy: 0.7119 - val_loss: 0.7602
Epoch 2/50
10221/10221 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.7053 - loss: 0.7833 - val_accuracy: 0.7254 - val_loss: 0.7348
Epoch 3/50
10221/10221 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.7167 - loss: 0.7580 - val_accuracy: 0.7318 - val_loss: 0.7171
Epoch 4/50
10221/10221 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.7200 - loss: 0.7461 - val_accuracy: 0.7338 - val_loss: 0.7110
Epoch 5/50
10221/10221 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.7232 - loss: 0.7436 - val_accuracy: 0.7375 - val_loss: 0.7024
Epoch 6/50
10221/10221 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.7265 - loss: 0.7316 - val_accuracy: 0.7412 - val_loss: 0.6968
Epoch 7/50
10221/10221 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.7296 - loss: 0.7276 - val_accuracy: 0.7417 - val_loss: 0.6934
Epoch 8/50
10221/10221 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 

In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load the validation dataset (ensure this path is correct)
telemetry_val = pd.read_csv('telemetry_for_operations_validation.csv')

# Check for missing columns
required_columns = ['mdm_object_name', 'mdm_model_id', 'lon', 'lat', 'alt', 
                    'speed_gps', 'direction', 'accel_forward_nn', 
                    'accel_braking_nn', 'accel_angular_nn', 
                    'accel_vertical_nn', 'speed_change', 'distance']

# Add the missing columns with default values (e.g., zeros)
for col in required_columns:
    if col not in telemetry_val.columns:
        telemetry_val[col] = 0.0  # Set default value, adjust if necessary

# Now you should have all required columns
print(telemetry_val.info())  # Verify the columns

# Prepare features for the model (drop create_dt and operation_kind_id)
X_valid = telemetry_val.drop(columns=['create_dt', 'operation_kind_id'], errors='ignore')

# If you applied any encoding to categorical variables in training, do the same here
# For example, if you used one-hot encoding for 'mdm_model_id':
# X_valid = pd.get_dummies(X_valid, columns=['mdm_model_id'], drop_first=True)

# Normalize the features (use the same scaler fitted on training data)
# Assuming you have saved your scaler from the training phase
# If not, you will need to fit the scaler again. Ideally, you should save the fitted scaler after training.
scaler = StandardScaler()
# Fit the scaler on the training data only (make sure you have access to the training data)
# scaler.fit(X_train)  # Uncomment this if you have the training data

# For now, let's scale the validation data directly
X_valid_scaled = scaler.fit_transform(X_valid)  # Replace this with scaler.transform(X_valid) if you have the scaler

# Make predictions using the trained model
predictions = model.predict(X_valid_scaled)  # Use the scaled validation data for predictions

# Since predictions from model.predict will be probabilities, get the class labels
predicted_classes = np.argmax(predictions, axis=1)
predicted_classes = np.where(predicted_classes == 4, 5, predicted_classes)
# Create the submission DataFrame
submission = pd.DataFrame({
    'create_dt': telemetry_val.get('create_dt', pd.Series(np.nan)),  # Include this column if needed
    'mdm_object_name': telemetry_val['mdm_object_name'],
    'operation_kind_id': predicted_classes
})

# Set the index as required
submission.set_index(['create_dt', 'mdm_object_name'], inplace=True)

# Ensure the shape is (expected_number_of_rows, 3)
submission.reset_index(drop=False, inplace=True)  # Reset index to maintain the specified columns

# Save the submission file
submission.to_csv('submission.csv', index=False)  # Save without index

# Optionally, display the first few rows of the submission
print(submission.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260111 entries, 0 to 260110
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   create_dt          260111 non-null  object 
 1   mdm_object_name    260111 non-null  int64  
 2   mdm_model_id       260111 non-null  int64  
 3   lon                260111 non-null  float64
 4   lat                260111 non-null  float64
 5   alt                260111 non-null  int64  
 6   speed_gps          260111 non-null  float64
 7   direction          260111 non-null  float64
 8   accel_forward_nn   260111 non-null  float64
 9   accel_braking_nn   260111 non-null  float64
 10  accel_angular_nn   260111 non-null  float64
 11  accel_vertical_nn  260111 non-null  float64
 12  speed_change       260111 non-null  float64
 13  distance           260111 non-null  float64
dtypes: float64(10), int64(3), object(1)
memory usage: 27.8+ MB
None
8129/8129 ━━━━━━━━━━━━━━━━━━━━ 6s 69

In [ ]:
## with data sampling 

In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

# Load your dataset
data = pd.read_csv('merged_telemetry_train.csv')  # Change this to your actual dataset path

# Data Cleaning
data.dropna(inplace=True)  # Drop rows with missing values

# Convert the target variable to int if necessary
data['operation_kind_id'] = data['operation_kind_id'].astype(int)
data['operation_kind_id'] = data['operation_kind_id'].replace(5, 4)

# Feature selection
features = ['mdm_object_name', 'mdm_model_id', 'lon', 'lat', 'alt', 
            'speed_gps', 'direction', 'accel_forward_nn', 
            'accel_braking_nn', 'accel_angular_nn', 
            'accel_vertical_nn', 'speed_change', 'distance']
X = data[features]
y = data['operation_kind_id']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f'Original dataset shape: {Counter(y_train)}')
print(f'Balanced dataset shape: {Counter(y_train_balanced)}')

# Normalize the features
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test_scaled = scaler.transform(X_test)

# Define the model
def create_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')  # Adjust for your number of classes
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_model(input_shape=(X_train_balanced.shape[1],))


Original dataset shape: Counter({2: 186975, 4: 133310, 0: 54279, 1: 28382, 3: 5882})
Balanced dataset shape: Counter({4: 186975, 2: 186975, 0: 186975, 1: 186975, 3: 186975})


In [41]:
# Train the model
from tensorflow.keras.callbacks import ModelCheckpoint
# Set up ModelCheckpoint to save the best model weights
checkpoint_filepath = 'best_model.weights.h5'  # Updated path
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',  # Monitor validation loss
    mode='min',  # Save the model with the lowest validation loss
    save_best_only=True  # Only save the best model
)



In [44]:
# Train the model with the checkpoint callback
model.fit(X_train_balanced, y_train_balanced, epochs=100, batch_size=32, validation_split=0.2,
          callbacks=[model_checkpoint_callback])

# Evaluate the model
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)

# Print the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))

Epoch 1/100
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 25s 1ms/step - accuracy: 0.7039 - loss: 0.8126 - val_accuracy: 0.5424 - val_loss: 1.2197
Epoch 2/100
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 25s 1ms/step - accuracy: 0.7042 - loss: 0.8117 - val_accuracy: 0.5919 - val_loss: 1.1199
Epoch 3/100
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 24s 1ms/step - accuracy: 0.7056 - loss: 0.8106 - val_accuracy: 0.5703 - val_loss: 1.2019
Epoch 4/100
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 26s 1ms/step - accuracy: 0.7062 - loss: 0.8099 - val_accuracy: 0.5772 - val_loss: 1.1882
Epoch 5/100
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 26s 1ms/step - accuracy: 0.7069 - loss: 0.8076 - val_accuracy: 0.5981 - val_loss: 1.1260
Epoch 6/100
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 25s 1ms/step - accuracy: 0.7057 - loss: 0.8097 - val_accuracy: 0.5901 - val_loss: 1.1235
Epoch 7/100
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 27s 1ms/step - accuracy: 0.7059 - loss: 0.8086 - val_accuracy: 0.5901 - val_loss: 1.1584
Epoch 8/100
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 28s 1ms/step - ac

In [45]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load the validation dataset (ensure this path is correct)
telemetry_val = pd.read_csv('telemetry_for_operations_validation.csv')

# Check for missing columns
required_columns = ['mdm_object_name', 'mdm_model_id', 'lon', 'lat', 'alt', 
                    'speed_gps', 'direction', 'accel_forward_nn', 
                    'accel_braking_nn', 'accel_angular_nn', 
                    'accel_vertical_nn', 'speed_change', 'distance']

# Add the missing columns with default values (e.g., zeros)
for col in required_columns:
    if col not in telemetry_val.columns:
        telemetry_val[col] = 0.0  # Set default value, adjust if necessary

# Now you should have all required columns
print(telemetry_val.info())  # Verify the columns

# Prepare features for the model (drop create_dt and operation_kind_id)
X_valid = telemetry_val.drop(columns=['create_dt', 'operation_kind_id'], errors='ignore')

# If you applied any encoding to categorical variables in training, do the same here
# For example, if you used one-hot encoding for 'mdm_model_id':
# X_valid = pd.get_dummies(X_valid, columns=['mdm_model_id'], drop_first=True)

# Normalize the features (use the same scaler fitted on training data)
# Assuming you have saved your scaler from the training phase
# If not, you will need to fit the scaler again. Ideally, you should save the fitted scaler after training.
scaler = StandardScaler()
# Fit the scaler on the training data only (make sure you have access to the training data)
# scaler.fit(X_train)  # Uncomment this if you have the training data

# For now, let's scale the validation data directly
X_valid_scaled = scaler.fit_transform(X_valid)  # Replace this with scaler.transform(X_valid) if you have the scaler

# Make predictions using the trained model
predictions = model.predict(X_valid_scaled)  # Use the scaled validation data for predictions

# Since predictions from model.predict will be probabilities, get the class labels
predicted_classes = np.argmax(predictions, axis=1)
predicted_classes = np.where(predicted_classes == 4, 5, predicted_classes)
# Create the submission DataFrame
submission = pd.DataFrame({
    'create_dt': telemetry_val.get('create_dt', pd.Series(np.nan)),  # Include this column if needed
    'mdm_object_name': telemetry_val['mdm_object_name'],
    'operation_kind_id': predicted_classes
})

# Set the index as required
submission.set_index(['create_dt', 'mdm_object_name'], inplace=True)

# Ensure the shape is (expected_number_of_rows, 3)
submission.reset_index(drop=False, inplace=True)  # Reset index to maintain the specified columns

# Save the submission file
submission.to_csv('submission.csv', index=False)  # Save without index

# Optionally, display the first few rows of the submission
print(submission.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260111 entries, 0 to 260110
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   create_dt          260111 non-null  object 
 1   mdm_object_name    260111 non-null  int64  
 2   mdm_model_id       260111 non-null  int64  
 3   lon                260111 non-null  float64
 4   lat                260111 non-null  float64
 5   alt                260111 non-null  int64  
 6   speed_gps          260111 non-null  float64
 7   direction          260111 non-null  float64
 8   accel_forward_nn   260111 non-null  float64
 9   accel_braking_nn   260111 non-null  float64
 10  accel_angular_nn   260111 non-null  float64
 11  accel_vertical_nn  260111 non-null  float64
 12  speed_change       260111 non-null  float64
 13  distance           260111 non-null  float64
dtypes: float64(10), int64(3), object(1)
memory usage: 27.8+ MB
None
8129/8129 ━━━━━━━━━━━━━━━━━━━━ 7s 89

## using class weighting 

In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE

# Load your dataset
data = pd.read_csv('merged_telemetry_train.csv')  # Adjust the path to your dataset

# Data cleaning and preparation
data.dropna(inplace=True)  # Drop rows with missing values
data['operation_kind_id'] = data['operation_kind_id'].astype(int)  # Ensure correct dtype
data['operation_kind_id'] = data['operation_kind_id'].replace(5, 4)
# Define features and target
features = ['mdm_object_name', 'mdm_model_id', 'lon', 'lat', 'alt', 
            'speed_gps', 'direction', 'accel_forward_nn', 'accel_braking_nn', 
            'accel_angular_nn', 'accel_vertical_nn', 'speed_change', 'distance']
X = data[features].values
y = data['operation_kind_id'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Normalize the features
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test_scaled = scaler.transform(X_test)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# Define the model
def create_model(input_shape):
    model = keras.Sequential([
        keras.layers.Input(shape=input_shape),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(len(np.unique(y)), activation='softmax')  # Adjust for number of classes
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create the model
model = create_model(input_shape=(X_train_balanced.shape[1],))

# Set up ModelCheckpoint to save the best model
checkpoint_filepath = 'best_model.keras'  # Path where the model will be saved
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,  # Save the entire model
    monitor='val_loss',  # Monitor validation loss
    mode='min',  # Save the model with the lowest validation loss
    save_best_only=True  # Only save the best model
)




In [50]:
# Train the model with class weights
model.fit(X_train_balanced, y_train_balanced, epochs=10, batch_size=32, 
          validation_split=0.2, class_weight=class_weights_dict,
          callbacks=[model_checkpoint_callback])

# Evaluate the model
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)

# Print the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))

Epoch 1/10
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 27s 1ms/step - accuracy: 0.3696 - loss: 2.2444 - val_accuracy: 0.7121 - val_loss: 0.8303
Epoch 2/10
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 26s 1ms/step - accuracy: 0.4821 - loss: 1.9583 - val_accuracy: 0.6920 - val_loss: 0.8496
Epoch 3/10
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 28s 1ms/step - accuracy: 0.5094 - loss: 1.8680 - val_accuracy: 0.7059 - val_loss: 0.8324
Epoch 4/10
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 24s 1ms/step - accuracy: 0.5203 - loss: 1.8316 - val_accuracy: 0.7098 - val_loss: 0.8125
Epoch 5/10
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 26s 1ms/step - accuracy: 0.5297 - loss: 1.7975 - val_accuracy: 0.7190 - val_loss: 0.8131
Epoch 6/10
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 26s 1ms/step - accuracy: 0.5413 - loss: 1.7632 - val_accuracy: 0.7137 - val_loss: 0.8116
Epoch 7/10
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 26s 1ms/step - accuracy: 0.5446 - loss: 1.7463 - val_accuracy: 0.7223 - val_loss: 0.7769
Epoch 8/10
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 26s 1ms/step - accuracy: 

In [51]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load the validation dataset (ensure this path is correct)
telemetry_val = pd.read_csv('telemetry_for_operations_validation.csv')

# Check for missing columns
required_columns = ['mdm_object_name', 'mdm_model_id', 'lon', 'lat', 'alt', 
                    'speed_gps', 'direction', 'accel_forward_nn', 
                    'accel_braking_nn', 'accel_angular_nn', 
                    'accel_vertical_nn', 'speed_change', 'distance']

# Add the missing columns with default values (e.g., zeros)
for col in required_columns:
    if col not in telemetry_val.columns:
        telemetry_val[col] = 0.0  # Set default value, adjust if necessary

# Now you should have all required columns
print(telemetry_val.info())  # Verify the columns

# Prepare features for the model (drop create_dt and operation_kind_id)
X_valid = telemetry_val.drop(columns=['create_dt', 'operation_kind_id'], errors='ignore')

# If you applied any encoding to categorical variables in training, do the same here
# For example, if you used one-hot encoding for 'mdm_model_id':
# X_valid = pd.get_dummies(X_valid, columns=['mdm_model_id'], drop_first=True)

# Normalize the features (use the same scaler fitted on training data)
# Assuming you have saved your scaler from the training phase
# If not, you will need to fit the scaler again. Ideally, you should save the fitted scaler after training.
scaler = StandardScaler()
# Fit the scaler on the training data only (make sure you have access to the training data)
# scaler.fit(X_train)  # Uncomment this if you have the training data

# For now, let's scale the validation data directly
X_valid_scaled = scaler.fit_transform(X_valid)  # Replace this with scaler.transform(X_valid) if you have the scaler

# Make predictions using the trained model
predictions = model.predict(X_valid_scaled)  # Use the scaled validation data for predictions

# Since predictions from model.predict will be probabilities, get the class labels
predicted_classes = np.argmax(predictions, axis=1)
predicted_classes = np.where(predicted_classes == 4, 5, predicted_classes)
# Create the submission DataFrame
submission = pd.DataFrame({
    'create_dt': telemetry_val.get('create_dt', pd.Series(np.nan)),  # Include this column if needed
    'mdm_object_name': telemetry_val['mdm_object_name'],
    'operation_kind_id': predicted_classes
})

# Set the index as required
submission.set_index(['create_dt', 'mdm_object_name'], inplace=True)

# Ensure the shape is (expected_number_of_rows, 3)
submission.reset_index(drop=False, inplace=True)  # Reset index to maintain the specified columns

# Save the submission file
submission.to_csv('submission.csv', index=False)  # Save without index

# Optionally, display the first few rows of the submission
print(submission.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260111 entries, 0 to 260110
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   create_dt          260111 non-null  object 
 1   mdm_object_name    260111 non-null  int64  
 2   mdm_model_id       260111 non-null  int64  
 3   lon                260111 non-null  float64
 4   lat                260111 non-null  float64
 5   alt                260111 non-null  int64  
 6   speed_gps          260111 non-null  float64
 7   direction          260111 non-null  float64
 8   accel_forward_nn   260111 non-null  float64
 9   accel_braking_nn   260111 non-null  float64
 10  accel_angular_nn   260111 non-null  float64
 11  accel_vertical_nn  260111 non-null  float64
 12  speed_change       260111 non-null  float64
 13  distance           260111 non-null  float64
dtypes: float64(10), int64(3), object(1)
memory usage: 27.8+ MB
None
8129/8129 ━━━━━━━━━━━━━━━━━━━━ 6s 74

## Ensemble appraoch 

In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint

# Load your dataset
data = pd.read_csv('merged_telemetry_train.csv')  # Adjust the path to your dataset

# Data cleaning and preparation
data.dropna(inplace=True)
data['operation_kind_id'] = data['operation_kind_id'].astype(int)
data['operation_kind_id'] = data['operation_kind_id'].replace(5, 4)
# Define features and target
features = ['mdm_object_name', 'mdm_model_id', 'lon', 'lat', 'alt', 
            'speed_gps', 'direction', 'accel_forward_nn', 'accel_braking_nn', 
            'accel_angular_nn', 'accel_vertical_nn', 'speed_change', 'distance']
X = data[features].values
y = data['operation_kind_id'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to create a model
def create_model(input_shape):
    model = keras.Sequential([
        keras.layers.Input(shape=input_shape),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(len(np.unique(y)), activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# List to hold predictions from each model
all_predictions = []

# Sampling with SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
scaler = StandardScaler()
X_train_smote = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

model_smote = create_model(input_shape=(X_train_smote.shape[1],))
model_smote.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2)
predictions_smote = model_smote.predict(X_test_scaled)
all_predictions.append(predictions_smote)

# Sampling with ADASYN
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)
X_train_adasyn = scaler.fit_transform(X_train_adasyn)

model_adasyn = create_model(input_shape=(X_train_adasyn.shape[1],))
model_adasyn.fit(X_train_adasyn, y_train_adasyn, epochs=15, batch_size=32, validation_split=0.2)
predictions_adasyn = model_adasyn.predict(X_test_scaled)
all_predictions.append(predictions_adasyn)

# Sampling with Random Undersampling
undersampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)
X_train_under = scaler.fit_transform(X_train_under)

model_under = create_model(input_shape=(X_train_under.shape[1],))
model_under.fit(X_train_under, y_train_under, epochs=15, batch_size=32, validation_split=0.2)
predictions_under = model_under.predict(X_test_scaled)
all_predictions.append(predictions_under)

# Ensemble the predictions (averaging)
ensemble_predictions = np.mean(all_predictions, axis=0)
ensemble_pred_classes = np.argmax(ensemble_predictions, axis=1)

# Print the confusion matrix and classification report
print(confusion_matrix(y_test, ensemble_pred_classes))
print(classification_report(y_test, ensemble_pred_classes))


Epoch 1/15
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 35s 1ms/step - accuracy: 0.5843 - loss: 1.0698 - val_accuracy: 0.4682 - val_loss: 1.3939
Epoch 2/15
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - accuracy: 0.6510 - loss: 0.9341 - val_accuracy: 0.4449 - val_loss: 1.4095
Epoch 3/15
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 33s 1ms/step - accuracy: 0.6637 - loss: 0.9046 - val_accuracy: 0.5513 - val_loss: 1.2506
Epoch 4/15
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - accuracy: 0.6699 - loss: 0.8901 - val_accuracy: 0.5149 - val_loss: 1.2437
Epoch 5/15
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 33s 1ms/step - accuracy: 0.6756 - loss: 0.8757 - val_accuracy: 0.5005 - val_loss: 1.3149
Epoch 6/15
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 34s 1ms/step - accuracy: 0.6781 - loss: 0.8705 - val_accuracy: 0.5123 - val_loss: 1.2973
Epoch 7/15
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - accuracy: 0.6805 - loss: 0.8641 - val_accuracy: 0.4996 - val_loss: 1.3335
Epoch 8/15
23372/23372 ━━━━━━━━━━━━━━━━━━━━ 35s 1ms/step - accuracy: 

In [55]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

# Load the validation dataset (ensure this path is correct)
telemetry_val = pd.read_csv('telemetry_for_operations_validation.csv')

# Check for missing columns
required_columns = ['mdm_object_name', 'mdm_model_id', 'lon', 'lat', 'alt', 
                    'speed_gps', 'direction', 'accel_forward_nn', 
                    'accel_braking_nn', 'accel_angular_nn', 
                    'accel_vertical_nn', 'speed_change', 'distance']

# Add the missing columns with default values (e.g., zeros)
for col in required_columns:
    if col not in telemetry_val.columns:
        telemetry_val[col] = 0.0  # Set default value, adjust if necessary

# Now you should have all required columns
print(telemetry_val.info())  # Verify the columns

# Prepare features for the model (drop create_dt and operation_kind_id)
X_valid = telemetry_val.drop(columns=['create_dt', 'operation_kind_id'], errors='ignore')

# Load the scaler from the training phase
scaler = StandardScaler()
# If you saved the scaler after fitting it on training data, load it here
# scaler = joblib.load('scaler.pkl')  # Example for loading a saved scaler

# Scale the validation data
X_valid_scaled = scaler.fit_transform(X_valid)  # Or use scaler.transform(X_valid) if you have the scaler

# Load the ensemble models (ensure you have the correct paths)
# model_smote = keras.models.load_model('model_smote.h5')
# model_adasyn = keras.models.load_model('model_adasyn.h5')
# model_under = keras.models.load_model('model_under.h5')

# Make predictions using each model
predictions_smote = model_smote.predict(X_valid_scaled)
predictions_adasyn = model_adasyn.predict(X_valid_scaled)
predictions_under = model_under.predict(X_valid_scaled)

# Average the predictions
ensemble_predictions = (predictions_smote + predictions_adasyn + predictions_under) / 3

# Get the predicted class labels
predicted_classes = np.argmax(ensemble_predictions, axis=1)
predicted_classes = np.where(predicted_classes == 4, 5, predicted_classes)  # Adjust for class 5

# Create the submission DataFrame
submission = pd.DataFrame({
    'create_dt': telemetry_val.get('create_dt', pd.Series(np.nan)),  # Include this column if needed
    'mdm_object_name': telemetry_val['mdm_object_name'],
    'operation_kind_id': predicted_classes
})

# Set the index as required
submission.set_index(['create_dt', 'mdm_object_name'], inplace=True)

# Ensure the shape is (expected_number_of_rows, 3)
submission.reset_index(drop=False, inplace=True)  # Reset index to maintain the specified columns

# Save the submission file
submission.to_csv('submission.csv', index=False)  # Save without index

# Optionally, display the first few rows of the submission
print(submission.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260111 entries, 0 to 260110
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   create_dt          260111 non-null  object 
 1   mdm_object_name    260111 non-null  int64  
 2   mdm_model_id       260111 non-null  int64  
 3   lon                260111 non-null  float64
 4   lat                260111 non-null  float64
 5   alt                260111 non-null  int64  
 6   speed_gps          260111 non-null  float64
 7   direction          260111 non-null  float64
 8   accel_forward_nn   260111 non-null  float64
 9   accel_braking_nn   260111 non-null  float64
 10  accel_angular_nn   260111 non-null  float64
 11  accel_vertical_nn  260111 non-null  float64
 12  speed_change       260111 non-null  float64
 13  distance           260111 non-null  float64
dtypes: float64(10), int64(3), object(1)
memory usage: 27.8+ MB
None
8129/8129 ━━━━━━━━━━━━━━━━━━━━ 7s 89